# CatBoost pipeline (basado en Modelo_prediccion)

Este notebook carga las funciones de `DataLoader.py`, prepara los datos respetando validación temporal, entrena un `CatBoostRegressor`, evalúa el modelo, calcula elasticidades y exporta las predicciones en formato Kaggle.

In [2]:
# Imports y funciones
import warnings
warnings.filterwarnings('ignore')
from DataLoader import importar_ventas, preparar_test, agregar_fourier, crear_archivo_kaggle, leer_csv
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import root_mean_squared_error, mean_absolute_error
import os
print('Imports OK')

Imports OK


In [3]:
# 1) Cargar datos (usa las funciones del DataLoader)

ventas = importar_ventas()

In [4]:
# 2) Añadir Fourier (k=2) si se pudo cargar

agregar_fourier(ventas, k=2)

print('Fourier agregado')

Fourier agregado


In [5]:
# 3) Definir features

cat_features = ['subgroup_cod','cluster']
num_features = ['max_price','min_price','year',
                'anual_sin_1','anual_cos_1','mensual_sin_1','mensual_cos_1']

# Categóricas (cat_features) → CatBoost las maneja de forma nativa.
# Numéricas (num_features) → precios, variables de tiempo, Fourier y lags

features = cat_features + num_features
print('Features definidos:', features)

Features definidos: ['subgroup_cod', 'cluster', 'max_price', 'min_price', 'year', 'anual_sin_1', 'anual_cos_1', 'mensual_sin_1', 'mensual_cos_1']


In [6]:
# 4) Split temporal: evitar leakage
if ventas is not None:
    max_date = ventas['date'].max()
    val_start = max_date - pd.Timedelta(days=7)
    
    train = ventas[ventas['date'] < val_start].copy()
    valid = ventas[ventas['date'] >= val_start].copy()
    
    print('Train dates:', train['date'].min(), '->', train['date'].max())
    print('Valid dates:', valid['date'].min(), '->', valid['date'].max())
else:
    train = valid = None

Train dates: 2021-01-01 00:00:00 -> 2023-12-23 00:00:00
Valid dates: 2023-12-24 00:00:00 -> 2023-12-31 00:00:00


In [7]:
# 5) Preparar matrices para CatBoost

if train is not None and valid is not None and len(valid)>0:
    
    X_train = train[features]
    y_train = train['demand'] * train['mean_price']
    
    X_valid = valid[features]
    y_valid = valid['demand'] * valid['mean_price']
    
    # Pools
    train_pool = Pool(X_train, y_train, cat_features=cat_features)
    valid_pool = Pool(X_valid, y_valid, cat_features=cat_features)
    
    print('Pools creados.')
    
else:
    train_pool = valid_pool = None
    print('No se pudieron crear pools por falta de datos')

Pools creados.


In [8]:
# 6) Entrenar CatBoost
if train_pool is not None and valid_pool is not None:
    model = CatBoostRegressor(
        iterations = 2000, # Número máximo de árboles de decisión
        learning_rate = 0.05, # Controla cuánto se ajusta el modelo en cada iteración
        depth = 8, # Profundidad máxima de cada árbol.
        loss_function = 'RMSE', # Función de pérdida que el modelo optimiza
        eval_metric = 'RMSE', # Métrica usada para evaluar el rendimiento en el conjunto de validación durante el entrenamiento
        random_seed = 42, # Semilla aleatoria para reproducibilidad.
        early_stopping_rounds = 100, # Si el modelo no mejora la métrica de validación en 100 iteraciones consecutivas, se detiene antes de llegar a iterations.
        verbose=100 # Imprime el progreso cada 100 iteraciones.
    )
    
    model.fit(train_pool, eval_set=valid_pool)
    model.save_model('catboost_model.cbm')
    print('Modelo entrenado y guardado: catboost_model_2.cbm')
    
else:
    model = None
    print('No se entrenó el modelo')

0:	learn: 541.2749955	test: 432.0585146	best: 432.0585146 (0)	total: 2.04s	remaining: 1h 8m 4s
100:	learn: 314.2336895	test: 238.4541843	best: 238.4541843 (100)	total: 2m 29s	remaining: 46m 43s
200:	learn: 299.7196660	test: 232.6594700	best: 232.6594700 (200)	total: 4m 54s	remaining: 43m 53s
300:	learn: 292.0080187	test: 230.4593270	best: 230.4593270 (300)	total: 7m 18s	remaining: 41m 14s
400:	learn: 287.5303210	test: 229.0589808	best: 229.0589808 (400)	total: 9m 38s	remaining: 38m 25s
500:	learn: 284.7707845	test: 228.3900723	best: 228.3900723 (500)	total: 11m 53s	remaining: 35m 33s
600:	learn: 282.2580904	test: 227.7269686	best: 227.7269686 (600)	total: 14m 10s	remaining: 33m
700:	learn: 279.8921675	test: 227.3253495	best: 227.3253495 (700)	total: 16m 23s	remaining: 30m 22s
800:	learn: 278.1361973	test: 226.9534986	best: 226.9449914 (797)	total: 18m 35s	remaining: 27m 49s
900:	learn: 277.0379747	test: 226.6384921	best: 226.6384921 (900)	total: 20m 45s	remaining: 25m 18s
1000:	learn: 

In [9]:
# 7) Evaluación en validación

if model is not None and valid is not None:
    
    y_pred_val = model.predict(valid[features])
    rmse = root_mean_squared_error(valid['demand'] * valid['mean_price'], y_pred_val)
    mae  = mean_absolute_error(valid['demand'] * valid['mean_price'], y_pred_val)
    
    print(f'Valid RMSE: {rmse:.4f}, MAE: {mae:.4f}')
    
else:
    print('No hay evaluación realizada')

Valid RMSE: 225.5738, MAE: 110.6898


In [11]:
# 8) Predicción sobre test preparado y preparación de Kaggle

test = preparar_test('ids_test.csv', ventas, features)
agregar_fourier(test, k=2)

# Asegurar columnas
for c in num_features:
    if c not in test.columns:
        test[c] = np.nan
    
test = test.sort_values(['subgroup_cod','store_cod','date']).reset_index(drop=True)
X_test = test[features].fillna(0)
test_pool = Pool(X_test, cat_features=cat_features)

# Predicción en $
y_test_pred = model.predict(test_pool)

# Guardar en formato Kaggle
pred_df = pd.DataFrame({
    'store_subgroup_date_id': test['store_subgroup_date_id'],
    'Total Sales': y_test_pred
})
pred_df.to_csv('prediccion_kaggle.csv', index=False)
print('Archivo prediccion_kaggle.csv creado con Total Sales directamente')

Archivo prediccion_kaggle.csv creado con Total Sales directamente
